# import

In [ ]:
!which python
import struct 
struct.calcsize("P")*8
# pd.set_option('display.max_rows', sub_pressure_df.shape[0]+1)

In [1]:
# !pip list
# !pip install openpyxl
#!pip install Shapely
#!pip install descartes
#!pip install hampel

You should consider upgrading via the 'c:\users\junec\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import rcParams
from operator import itemgetter
from typing import Callable, Dict, List, Set, Tuple
from scipy.optimize import curve_fit
import csv
import json
from collections import defaultdict
from scipy.signal import savgol_filter
import math 
import warnings
warnings.simplefilter('ignore', np.RankWarning)
from pprint import pprint
import os

In [1]:
import sys
sys.path.insert(1, 'methods')
sys.path.insert(1, 'util')
from util import SelectRows,calculate_derivative,pointInterval_to_pressure,point_dt_to_pressure,print_tuning_parameters,timeInterval_to_sub_df
from baseline_method import detect_breakpoints_startPoint, detect_breakpoints, detect_breakpoints_2, detect_breakpoints_3,detect_breakpoints_4,detect_breakpoints_startPoint2
from plot import PlotNSave
from plot2 import PlotNSave
from data_load_N_preprocess import LoadNPreprocessData
from base_classes import CurveParametersCalc
from patternRecognition_method import PatternRecognitionMethod
from tangent_method import TangentMethod
# from advanced_method import detect_max_FOD
from derivative_method import DerivativeMethod
from store_transients import StoreTransients
# from patternRecognition_method_N_tangent_method import PatternRecognition


# load data and process

## synthetic

In [3]:
#experiment setup
print("load&process Synthetic data===========================")
ground_truth_synthetic=[107, 164, 243, 328, 375, 446, 644, 713, 771, 820, 887, 953, 1036, 1101, 1179, 1254, 1319, 1396, 1456, 1513, 1598, 1648, 1719, 1784, 1875, 1938, 2017, 2073, 2185, 2247, 2315, 2369, 2433, 2487, 2552, 2610, 2835, 2916, 2965, 3044, 3101, 3240, 3284, 3368, 3411]
print("len(ground_truth_synthetic):",len(ground_truth_synthetic))
colum_names_synthetic={"pressure":{"time":"Date",
                        "measure":"Pressure (psia)",
                        "first_order_derivative":"first_order_derivative",
                        "second_order_derivative":"second_order_derivative"},
             "rate":{"time":"Time@end",
                     "measure":"Liquid rate (STB/D)"}}
data_inOneRow_synthetic=400
pressure_filePath="../data_input/Synthetic_Data.xlsx"
rate_filePath="../data_input/Synthetic_Data.xlsx"

processed_data=LoadNPreprocessData(pressure_filePath=pressure_filePath, 
                 rate_filePath=rate_filePath, 
                 colum_names=colum_names_synthetic, 
                 use_SG_smoothing=False)
pressure_df_synthetic=processed_data.pressure_df
rate_df_synthetic=processed_data.rate_df

print("len(pressure_df_synthetic)",len(pressure_df_synthetic))
print("len(rate_df_synthetic)",len(rate_df_synthetic))

pressure_measure_synthetic=list(pressure_df_synthetic[colum_names_synthetic["pressure"]["measure"]])
pressure_time_synthetic=list(pressure_df_synthetic[colum_names_synthetic["pressure"]["time"]])

len(ground_truth_synthetic): 45
---load data from 'txt' or 'xlsx' files...
---The first & second order derivative has been calculated and appended to pressure dataframe
len(pressure_df_synthetic) 3519
len(rate_df_synthetic) 95


## denoised data

In [4]:
#experiment setup
print("load & denoise real field data===========================")
ground_truth_temp=[2249,  2467,  2805,[2875, 2876],3100,3561,3994,[4228, 4232],4753,4895, 5297,5436,5985,[6543, 6547], 7249,[7544, 7545],7946,[8362, 8364],9025,[9607, 9608],10149,10216, 10528,[10801, 10804],11340,  11479,12363,[13039, 13040],13868, 14086,14774,15341,16098,16619,[17589, 17590],18435,19643,19764,20021,20349,20701,21030,21974,22186,22710,23220,24215,24685,25717,26006,26656, 26759,27023,[27316,27327],27773,[27966,27970],28322,28497,28832,29115]
colum_names_denoised   ={"pressure":{"time":"Elapsed time",
                             "measure":"Data",
                             "first_order_derivative":"first_order_derivative",
                             "second_order_derivative":"second_order_derivative"},
                "rate":{"time":"Elapsed time",
                        "measure":"Liquid rate"}}
data_inOneRow_denoised=1200
pressure_filePath="../data_input/EAGE_Shchipanov_2017/Pressure.txt"
rate_filePath="../data_input/EAGE_Shchipanov_2017/Rate.txt"
ground_truth=[]
for point in ground_truth_temp: 
    if "list" in str(type(point)):
        ground_truth.append(math.ceil(sum(point)/len(point)))
    else:
        ground_truth.append(point)
                     

processed_data_denoised=LoadNPreprocessData(pressure_filePath=pressure_filePath, 
                 rate_filePath=rate_filePath, 
                 colum_names=colum_names_denoised, 
                 use_SG_smoothing=True)
pressure_df_denoised=processed_data_denoised.pressure_df
rate_df_denoised=processed_data_denoised.rate_df

pressure_df=pressure_df_denoised
pressure_measure_denoised=list(pressure_df[colum_names_denoised["pressure"]["measure"]])
print("len(pressure_measure_denoised)",len(pressure_measure_denoised))
pressure_time_denoised=list(pressure_df[colum_names_denoised["pressure"]["time"]])
second_derivatives=list(pressure_df[colum_names_denoised["pressure"]["second_order_derivative"]])
# ground truth adapted for denoise data
ground_truth_denoised=[2248, 2467, 2804, 2884, 3095, 3560, 3987, 4246, 4750, 4899, 5297, 5436, 5983, 6543, 7249, 7545, 7946, 8361, 9024, 9619, 10149, 10224, 10527, 10800, 11340,11490, 12362, 13040, 13867, 14089, 14766, 15348, 16097, 16619, 17589, 18434, 19642, 19777, 20014, 20358, 20700, 21025, 21974, 22183, 22709, 23219, 24215, 24683, 25698, 26011, 26656, 26753, 27022, 27315, 27760, 27969, 28322, 28494, 28831, 29115]
print("len(ground_truth_denoised)",len(ground_truth_denoised))
ground_truth_badQuality=[9024,10800,2467,10224,21025,26753]
for item in ground_truth_badQuality:
    ground_truth_denoised.remove(item)

print("after remove uncorrect ground truth")
print("len(ground_truth_denoised)",len(ground_truth_denoised))

---load data from 'txt' or 'xlsx' files...
check the time type
---denoising data using S-G smoothing...
---The first & second order derivative has been calculated and appended to pressure dataframe
len(pressure_measure_denoised) 29813
len(ground_truth_denoised) 60
after remove uncorrect ground truth
len(ground_truth_denoised) 54


In [ ]:
pressure_measure=pressure_measure_denoised
pressure_time=pressure_time_denoised
pressure_df=pressure_df_denoised
# pressure_measure=pressure_measure_denoised[0:3500]
# pressure_time=pressure_time_denoised[0:3500]
# pressure_df=pressure_df_denoised[0:3500]
ground_truth=ground_truth_denoised
rate_df=rate_df_denoised
colum_names=colum_names_denoised
data_inOneRow=data_inOneRow_denoised